# Data Collection and Storage

This notebook implements describes the collection and storage of data used for the project application.

Contents
--------
1. [Fetching data through API calls](#api)
    1. [Get geocoding information](#geocoding)
    2. [Get weather forecast](#weather)
2. [Web scraping](#scraping)
    1. [Reverse engineering the website requests](#scraping_reverse)
    2. [Get hotel information by scraping](#scraping_fetch)
3. [Storage in a database](#database)
4. [Storage in a data lake](#datalake)


## <a name="api"></a>API Calls

We request APIs to get geocoding information from the name of a place and weather forecast at given geographic coordinates. Utilities for the corresponding API calls are defined in the module `etl/api_mgmt.py`.

To use them, we first setup a `requests.Session`. Since the API servers may limit the number of allowed requests, we add a retry policy to the HTTP session. We also load a file containing the names of the locations of interest.

In [1]:
import csv
import json
import time

import numpy as np
import requests
from requests.adapters import HTTPAdapter, Retry

from etl import get_coords, get_weather_forecast, save_to_json

In [2]:
# setup session with retry policy in case of failure
s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[403, 502, 503, 504])
s.mount('https://', HTTPAdapter(max_retries=retries))

# Load locations of interest
with open("./data/places.csv", 'rt', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader, None) # remove header
    locations = [row for row in reader]

### <a name="geocoding"></a>Get geocoding information

We use [Nominatim API](https://nominatim.org/) to fetch geocoding information. The API is quite restrictive in its [use policy](https://operations.osmfoundation.org/policies/nominatim/). Most importantly, the number of requests is limited to one per second, which forces us to throttle the rate accordingly.

In [3]:
coordinates = {}
for i, loc in enumerate(locations):
    coordinates[i] = ({'place': loc[0], 'country': loc[1]} 
                      | get_coords(s, f"{loc[0]}, {loc[1]}"))
    time.sleep(1.1)

coordinates[4]

KeyboardInterrupt: 

In [ ]:
# save to csv
locations_cols = ['location_id', 'place', 'country', 'latitude', 'longitude']
with open('./data/locations.csv', 'wt', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writeheader(locations_cols)
    for i, coords in coordinates:
        writer.writerow([i] + [coords[col] for col in locations_cols[1:]])

### <a name="weather"></a>Get weather forecast

We use [Open-Meteo API](https://open-meteo.com/en/docs) to get weather forecast information.

In [ ]:
weather_forecast = {}
for i, coords in coordinates.items():
    weather_forecast[i] = get_weather_forecast(
        s, coords['latitude'], coords['longitude'])
save_to_json('./data/temp/weather_forecast.json', weather_forecast)

weather_forecast[4]

{'latitude': 48.86,
 'longitude': 2.3199997,
 'generationtime_ms': 0.14066696166992188,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/Paris',
 'timezone_abbreviation': 'GMT+1',
 'elevation': 38.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_max': '°C',
  'temperature_2m_min': '°C',
  'sunshine_duration': 's',
  'precipitation_sum': 'mm'},
 'daily': {'time': ['2025-02-23',
   '2025-02-24',
   '2025-02-25',
   '2025-02-26',
   '2025-02-27',
   '2025-02-28',
   '2025-03-01',
   '2025-03-02',
   '2025-03-03',
   '2025-03-04',
   '2025-03-05',
   '2025-03-06',
   '2025-03-07',
   '2025-03-08',
   '2025-03-09',
   '2025-03-10'],
  'temperature_2m_max': [14.6,
   11.3,
   10.1,
   10.6,
   10.0,
   9.8,
   10.4,
   10.6,
   8.8,
   11.1,
   11.3,
   11.9,
   14.9,
   11.6,
   8.9,
   6.2],
  'temperature_2m_min': [4.8,
   9.2,
   5.2,
   3.7,
   3.6,
   0.2,
   1.0,
   1.1,
   4.9,
   4.2,
   8.8,
   10.0,
   10.0,
   6.7,
   5.2,
   5.5],
  'sunshine_duration': [25805.69,
   0.

In [ ]:
# save to csv
# take the average weather over the next 7 days
weather_cols = [
    'location_id', 'date', 'min_temperature_C', 'max_temperature_C',
    'sunshine_duration_h', 'precipitation_sum_mm'
]
with open('./data/weather_indicators.csv', 'wt', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writeheader(weather_cols)
    for i, forecast in weather_forecast.items():
        forecast = forecast['daily']
        row = [i, forecast['time'][0],
               np.mean(forecast['temperature_2m_min'][:8]),
               np.mean(forecast['temperature_2m_max'][:8]),
               np.mean(forecast['sunshine_duration'][:8])/3600,
               np.mean(forecast['precipitation_sum'][:8])]
        writer.writerow(row)

## <a name="scraping"></a>Web scraping

The collection of hotels information at the selected locations is done through web scraping of [booking.com](https://www.booking.com). This approach is more complex and unstable than API calls. It is standard practice to first study how requests and responses are related to web browser interaction. This will allow us to tailor automated requests for scraping.

### <a name="scraping_reverse"></a>Reverse engineering the website requests

The first step is to get the request resulting from regular user interaction. We thus go to the index page [https://www.booking.com/index.en-gb.html](https://www.booking.com/index.en-gb.html) and fill the search bar. Here we look for an hotal in Rouen, France for 2 adults between march, 1st and march, 9th.

<img src="media/booking_search.png" alt="booking_search" width="1000"/>

After clicking on "Search" the request actually sent by the web browser is displayed in the address bar. This allows us to recover the parameters of the GET request.

In [ ]:
req_url = 'https://www.booking.com/searchresults.en-gb.html?ss=Rouen%2C+France&efdco=1&label=gen173nr-1FCAEoggI46AdICVgEaE2IAQGYAQm4ARjIAQ_YAQHoAQH4AQKIAgGoAgS4AsWf4r0GwAIB0gIkYmMwNmI1MjktMzkyZS00N2FjLTllNWYtOWZmZGIwMWZjODhj2AIF4AIB&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=-1462807&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=a6f166e2b250077a&ac_meta=GhBhNmYxNjZlMmIyNTAwNzdhIAAoATICZW46DVJvdWVuLCBGcmFuY2VAAEoAUAA%3D&checkin=2025-03-01&checkout=2025-03-09&group_adults=2&no_rooms=1&group_children=0'

req_url.split('?')[1].split('&')

['ss=Rouen%2C+France',
 'efdco=1',
 'label=gen173nr-1FCAEoggI46AdICVgEaE2IAQGYAQm4ARjIAQ_YAQHoAQH4AQKIAgGoAgS4AsWf4r0GwAIB0gIkYmMwNmI1MjktMzkyZS00N2FjLTllNWYtOWZmZGIwMWZjODhj2AIF4AIB',
 'aid=304142',
 'lang=en-gb',
 'sb=1',
 'src_elem=sb',
 'src=index',
 'dest_id=-1462807',
 'dest_type=city',
 'ac_position=0',
 'ac_click_type=b',
 'ac_langcode=en',
 'ac_suggestion_list_length=5',
 'search_selected=true',
 'search_pageview_id=a6f166e2b250077a',
 'ac_meta=GhBhNmYxNjZlMmIyNTAwNzdhIAAoATICZW46DVJvdWVuLCBGcmFuY2VAAEoAUAA%3D',
 'checkin=2025-03-01',
 'checkout=2025-03-09',
 'group_adults=2',
 'no_rooms=1',
 'group_children=0']

We can already get a few insights from the request URL:
- `ss` (search string) corresponds to the text written in the search bar,
- `checkin` and `checkout` correspond to the travel dates (the calendar widget in the center),
- `group_adults`, `no_rooms` and `group_children` correspond to the input of the right widget.

It turns out that a valid request can be made with a different approach, by specifying only the latitude and longitude of the destination. For instance, the URL
`'https://www.booking.com/searchresults.en-gb.html?latitude=49.4404591&longitude=1.0939658'` yields a page with the hotels ranked by inreasing distance to the coordinates. The parameters above can also be specified to refine the search, but we will not use them here.

### <a name="scraping_fetch"></a>Get hotel information by scraping

The above analysis helped to setup the scraping functionality, which is implemented in the module `etl/scraping_mgmt.py`. Let us detail briefly the sraping procedure:
- We reach the target website through an automated browser driven with Selenium WebDriver. We favor this approach over other possibilities such as using `scrapy`. The reason is that our target, booking.com, implements infinite scrolling. This feature is implemented in javascript and is complex to trigger if the scraping tool used cannot execute javascript. A browser, however, natively executes javascript and therefore suits better our task.
- For each location, we send a request with the URL `'https://www.booking.com/searchresults.en-gb.html?latitude={latitude}&longitude={longitude}'`.
- We scrape the hotels data that we need, scrolling down if necessary.

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException

from etl import scrape_from_searchpage

In [ ]:
## setup driver with options to prevent detection
## See https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec/53040904#53040904
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd(
    'Network.setUserAgentOverride',
    {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'}
)
driver.implicitly_wait(1)

In [ ]:
with open("./data/locations.csv", 'rt', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader, None) # remove header
    locations = [row for row in reader]

search_urls = {loc[0]: ('https://www.booking.com/searchresults.en-gb.html?'
                        f'latitude={loc[3]}&longitude={loc[4]}')
               for loc in locations}

for i, search_url in search_urls.items():
    hotel_infos = {i: scrape_from_searchpage(driver, search_url, limit=30)} # scrape on search
    save_to_json(f'./data/temp/hotels/{i}.json', hotel_infos)
    break

driver.quit()

AttributeError: 'set' object has no attribute 'items'

In [ ]:
# load temporary saved files
hotels_list = []
root, _, files = next(os.walk('../data/temp/hotels/'))
for file in files:
    with open(root + file, 'rt', encoding='utf-8') as f:
        hotels_list.append(json.load(f))

# transform and save csv
hotels_cols = ['hotel_id', 'location_id', 'url', 'name',
               'description', 'rating', 'georating']
hotel_id = 1
hotels_data = []
for hotels in hotels_list:
    loc_id, hotels = next(iter(hotels.values()))
    for h in hotels:
        entry = [hotel_id, loc_id] + [h[col] for col in hotels_cols[2:]]
        hotel_id += 1

with open('../data/hotels.csv', 'wt', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writeheader(hotels_cols)
    for row in hotels_data:
        writer.writerow(row)

## <a name="database"></a>Storage in a database

## <a name="database"></a>Storage in a database

For long-term storage, the collected data is transferred a data lake in csv format. We use an AWS S3 bucket for that purpose. The functionality to transfer and load data from the database is implemented in the module `etl/db_mgmt.py`. We use SQLAlchemy to define our database structure, connected to a PostgreSQL database hosted on Neon. The structure of the database is the following.

<img src="media/DB_structure.png" alt="Database structure" width="700"/>

## <a name="datalake"></a>Storage in a data lake

For long-term storage, the collected data is transferred a data lake in csv format. We use an AWS S3 bucket for that purpose. The functionality to transfer and load data from the data lake is implemented in the module `etl/s3_mgmt.py`. We format our data in multiple csv file matching the database structure before uploading in the S3 bucket. A copy of the files is then downloaded and stored locally.